In [ ]:
import os
import sys
import pandas as pd
import re
from pdfminer.high_level import extract_text
from transformers import BertTokenizer, BertForNextSentencePrediction
import torch
%cd /home/kai/muku_esg/ESG-BERT

In [ ]:
def preprocess(x):
    x = x.replace('\n', '')
    x = re.sub(r'[!\"#$%&\'()*\+,-.\/:;<=>?@\[\\\]^_`{|}~]', '', x)
    x = x.replace('\t', ' ')
    if len(x.split()) < 5:
        return None
    else:
        return x

In [ ]:
def ascii(text):
    text = re.sub(r'[^\x00-\x7F]', '', text)
    text = re.sub(r'[\x0C]', r"\n\n", text)
    return text

In [ ]:
file_name = './taiwan_pdf/2_tsmc_1_210.pdf'
# text = extract_text(file_name, page_numbers=[i for i in range(15)])
text = extract_text(file_name, page_numbers=[11])
text = ascii(text)
text = text.strip('\"')
paragraph = re.split(r'\n\s*\n', text)
paragraph = map(preprocess, paragraph)
paragraph = [x for x in paragraph if x is not None]

In [ ]:
print(len(paragraph))
for i in range(len(paragraph)):
    print(paragraph[i])
    print()

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForNextSentencePrediction.from_pretrained("bert-base-uncased")

In [ ]:
paragraph = ['To fulfill our commitment to our Diversity and Inclusion Statement', ', the Company treats its workforce as the most important asset and is \
committed to empowering more than 60,000 employees globally to unleash their full potential in the right roles',
'In supporting employees to balance family life and career development, TSMC offers a welfare system that is better than legal \
requirements and has established four preschool for fabs in Taiwan',
'Weather is the state of the atmosphere, describing for example the degree to which it is hot or cold, wet or dry, calm or stormy, clear or cloudy.',
'On Earth, most weather phenomena occur in the lowest layer of the planets atmosphere, the troposphere, just below the stratosphere',
'Weather refers to day-to-day temperature, precipitation, and other atmospheric conditions, whereas climate is the term for the averaging of atmospheric conditions over longer periods of time.'
]

In [ ]:
ind = 0
while ind < len(paragraph) - 1:
    # if len(paragraph[ind].split()) + len(paragraph[ind+1].split()) >= 500:
    #     ind += 1
    prompt = paragraph[ind]
    next_sentence = paragraph[ind+1]
    print('prompt', prompt)
    print('next_sentence', next_sentence)
    encoding = tokenizer(prompt, next_sentence, truncation=True, return_tensors="pt")
    outputs = model(**encoding, labels=torch.LongTensor([1]))
    logits = outputs.logits
    print(logits)
    if logits[0, 0] > logits[0, 1]:  # next sentence was random
        paragraph[ind:ind+2] = [' '.join(paragraph[ind:ind+2])]
    else:
        ind += 1
    if len(paragraph[ind].split()) >= 500:
        ind += 1
print(len(paragraph))
print(paragraph)